# Snowflake Data Transfer Cost SQL Queries — Versión para **Snowflake Notebooks**
**Fecha:** 2025-11-06
**Autor:** Adolfo Orozco
**Tags:** #snowflake #snowpark #python #sql #cost #performance #recipes

Este cuaderno está optimizado para ejecutarse dentro de **Snowflake Notebooks** usando **Snowpark**. 
Cuando se ejecute fuera de Snowflake Notebooks (por ejemplo, en tu Jupyter local), hará *fallback* a una conexión estándar con credenciales.


## 1. Requisitos previos

- En **Snowflake Notebooks** no necesitas credenciales: se usa `get_active_session()`.
- Si corres localmente:
  - Paquetes: `snowflake-snowpark-python[pandas]`, `python-dotenv` (opcional).
  - Credenciales de Snowflake (account, user, password/SSO/TOTP, role, warehouse, database, schema).


In [ ]:
# 1.1 (Opcional, ejecución local) Instalar dependencias
# %pip install --quiet "snowflake-snowpark-python[pandas]" python-dotenv


## 2. Conexión con Snowflake Notebooks (Snowpark)

- En Snowflake Notebooks: `get_active_session()` devuelve una sesión lista para usar.
- En local: `Session.builder.configs({...}).create()` con tus credenciales.


In [ ]:
# 2.1 Crear/obtener la sesión de Snowpark
from __future__ import annotations

import os
import getpass

try:
    # Disponible dentro de Snowflake Notebooks
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark import Session
    session = get_active_session()
    IN_SNOWFLAKE_NOTEBOOK = True
except Exception:
    # Fallback local
    from snowflake.snowpark import Session
    IN_SNOWFLAKE_NOTEBOOK = False

def get_session() -> "Session":
    """Devuelve una sesión de Snowpark. Usa la sesión activa si estamos en Snowflake Notebooks.
    Si se ejecuta localmente, crea una sesión con credenciales proporcionadas por variables de entorno o input seguro."""
    global session
    if IN_SNOWFLAKE_NOTEBOOK:
        return session
    
    # Lectura de credenciales para ejecución local
    connection_parameters = {
        "account":   os.getenv("SNOWFLAKE_ACCOUNT", "your_account"),
        "user":      os.getenv("SNOWFLAKE_USER", "your_user"),
        "password":  os.getenv("SNOWFLAKE_PASSWORD") or getpass.getpass("Snowflake password: "),
        "role":      os.getenv("SNOWFLAKE_ROLE", "ACCOUNTADMIN"),
        "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH"),
        "database":  os.getenv("SNOWFLAKE_DATABASE", "SNOWFLAKE"),
        "schema":    os.getenv("SNOWFLAKE_SCHEMA", "ACCOUNT_USAGE"),
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

# Probar conexión
_sess = get_session()
_sess.sql("SELECT CURRENT_VERSION() AS VERSION").to_pandas()


## 3. Utilidades y contexto de sesión
Incluimos un *helper* para ejecutar SQL con Snowpark y devolver `pandas.DataFrame`, 
junto con una consulta para inspeccionar el contexto actual.


In [ ]:
# 3.1 Helper para ejecutar SQL y devolver pandas DataFrame
import pandas as pd

def run_sql_df(sql: str) -> pd.DataFrame:
    s = get_session()
    return s.sql(sql).to_pandas()

# 3.2 Contexto actual
ctx_sql = '''
SELECT CURRENT_ROLE() AS ROLE,
       CURRENT_WAREHOUSE() AS WAREHOUSE,
       CURRENT_DATABASE() AS DATABASE,
       CURRENT_SCHEMA() AS SCHEMA;
'''
run_sql_df(ctx_sql)


## 4. Carga vs. colas por warehouse (últimos 30 días)

**Propósito:** identificar por día la suma de consultas promedio en ejecución vs. en cola. 
Días con colas elevadas pueden sugerir activar**Multi-Cluster** o ajustar tamaños.


In [ ]:
# 4.1 Carga diaria vs. colas (últimos 30 días)
sql_load_vs_queue = '''
SELECT
    CAST(START_TIME AS DATE) AS DATE,
    WAREHOUSE_NAME,
    SUM(AVG_RUNNING)      AS SUM_RUNNING_QUERIES,
    SUM(AVG_QUEUED_LOAD)  AS SUM_QUEUED_QUERIES
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_LOAD_HISTORY
WHERE CAST(START_TIME AS DATE) >= DATEADD(MONTH, -1, CURRENT_DATE())
GROUP BY 1, 2
HAVING SUM(AVG_QUEUED_LOAD) > 0
ORDER BY DATE DESC, WAREHOUSE_NAME;
'''
run_sql_df(sql_load_vs_queue)
